<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

Import scopes are divided in several notebooks:
- [Calculation of Identities](../Import/1ImportScope-Identities)
- [Calculation of Present Values](../Import/2ImportScope-PresentValue)
- [Calculation of Actuals](../Import/3ImportScope-Actuals)
- [Calculation of Technical Margin](../Import/4ImportScope-TechnicalMargin)
- Creation of Ifrs Variables
- [Calculation of Ifrs Variables](../Import/6ImportScope-Compute)

<br><br>
In this notebook the focus is on the creation of Ifrs Variable from the Scopes.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "4ImportScope-TechnicalMargin"

# Computing All Scopes

Since all the calculations defined in the various scopes are to be performed together, it is useful to introduce the scopes: <code>ComputeIfrsVarsActuals</code>, <code>ComputeIfrsVarsCashflows</code> and the <code>ComputeIfrsVarsOpenings</code> below, which given a certain Import Format and Import Storage, performs all the calculations (defined above) based on the corresponding data:

## Pv

In [0]:
public interface PvToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
  static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PvToIfrsVariable>(s => s.WithApplicability<EmptyPvIfrsVariable>(x => ComputationHelper.AocStepWithNoPv.Contains(new AocStep(x.Identity.AocType, x.Identity.Novelty))));

  IEnumerable<IfrsVariable> PvLocked => GetScope<PvLocked>(Identity).PresentValues.Select(x => 
    new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                      EstimateType = x.Identity.EstimateType, 
                      DataNode = x.Identity.Id.DataNode, 
                      AocType = x.Identity.Id.AocType, 
                      Novelty = x.Identity.Id.Novelty, 
                      AccidentYear = x.Identity.AccidentYear,
                      AmountType = x.Identity.AmountType,
                      Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                      Partition = GetStorage().TargetPartition });
  IEnumerable<IfrsVariable> PvCurrent => GetScope<PvCurrent>(Identity).PresentValues.Select(x => 
    new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                      EstimateType = x.Identity.EstimateType, 
                      DataNode = x.Identity.Id.DataNode, 
                      AocType = x.Identity.Id.AocType, 
                      Novelty = x.Identity.Id.Novelty, 
                      AccidentYear = x.Identity.AccidentYear,
                      AmountType = x.Identity.AmountType,
                      Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                      Partition = GetStorage().TargetPartition });
}

public interface EmptyPvIfrsVariable: PvToIfrsVariable{
  IEnumerable<IfrsVariable> PvToIfrsVariable.PvLocked => Enumerable.Empty<IfrsVariable>();
  IEnumerable<IfrsVariable> PvToIfrsVariable.PvCurrent => Enumerable.Empty<IfrsVariable>();
}


## Nominal

In [0]:
public interface NominalToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> CumulatedNominal => GetScope<CumulatedNominalBE>(Identity).PresentValues
    .Concat(GetScope<CumulatedNominalRA>(Identity).PresentValues)
    .Select(x => new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod), 
                        Partition = GetStorage().TargetPartition });
}

## RiskAdjustment

In [0]:
public interface RaToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> RaCurrent => GetScope<RaCurrent>(Identity).PresentValues.Select(x => 
      new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = null,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
                        
    IEnumerable<IfrsVariable> RaLocked => GetScope<RaLocked>(Identity).PresentValues.Select(x => 
      new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                        EstimateType = x.Identity.EstimateType, 
                        DataNode = x.Identity.Id.DataNode, 
                        AocType = x.Identity.Id.AocType, 
                        Novelty = x.Identity.Id.Novelty, 
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = null,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
}

## Actuals

In [0]:
public interface ActualToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> Actual => GetScope<Actual>(Identity).Actuals.Select(x => 
      new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                        DataNode = x.Identity.Id.DataNode,
                        AocType = x.Identity.Id.AocType,
                        Novelty = x.Identity.Id.Novelty,
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
                        
    IEnumerable<IfrsVariable> AdvanceActual => GetScope<AdvanceActual>(Identity).Actuals.Select(x => 
      new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                        DataNode = x.Identity.Id.DataNode,
                        AocType = x.Identity.Id.AocType,
                        Novelty = x.Identity.Id.Novelty,
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });

   IEnumerable<IfrsVariable> OverdueActual => GetScope<OverdueActual>(Identity).Actuals.Select(x => 
      new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                        DataNode = x.Identity.Id.DataNode,
                        AocType = x.Identity.Id.AocType,
                        Novelty = x.Identity.Id.Novelty,
                        AccidentYear = x.Identity.AccidentYear,
                        AmountType = x.Identity.AmountType,
                        Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                        Partition = GetStorage().TargetPartition });
}

## Deferrable

In [0]:
public interface DeferrableToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    protected string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod); 

    IEnumerable<IfrsVariable> Deferrable => EconomicBasis switch {
        EconomicBases.N => Enumerable.Range(0, timeStep).SelectMany(shift => 
            GetScope<NominalDeferrable>((Identity, shift)).RepeatOnce()
                .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                          EconomicBasis = EconomicBases.N,
                          DataNode = x.Identity.Id.DataNode,
                          AocType = x.Identity.Id.AocType,
                          Novelty = x.Identity.Id.Novelty,
                          AccidentYear = shift,
                          Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition })),
        _ => GetScope<DiscountedDeferrable>(Identity).RepeatOnce()
                .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                          EconomicBasis = x.EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition }),
    };

    private IEnumerable<IfrsVariable> amortizationStep => Deferrable.Where(iv => iv.Values != null).Where(iv => Math.Abs(iv.Values.GetValidElement(Identity.ProjectionPeriod)) > Precision);

    IEnumerable<IfrsVariable> DeferrableAmFactor => (Identity.AocType, amortizationStep.Any(), EconomicBasis) switch {
        (AocTypes.AM, true, EconomicBases.N) => amortizationStep.Select(x => x.AccidentYear.Value).SelectMany(shift => 
            GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.DAE, shift), o => o.WithContext(EconomicBases.N)).RepeatOnce() //hardcoded AmountType: DAE for pattern
                .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                          EconomicBasis = EconomicBases.N,
                          DataNode = x.Identity.Id.DataNode,
                          AocType = Identity.AocType,
                          Novelty = Identity.Novelty,
                          AmountType = x.EffectiveAmountType,
                          AccidentYear = shift,
                          Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition })),
        (AocTypes.AM, true, _) => GetScope<DiscountedAmortizationFactorForDeferrals>(Identity, o => o.WithContext(EconomicBasis)).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = EstimateTypes.F,
                          EconomicBasis = EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          AmountType = x.EffectiveAmountType,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition }),
        (_) => Enumerable.Empty<IfrsVariable>(),
    };
}

## Premium Revenues

In [0]:
public interface RevenueToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<RevenueToIfrsVariable>(s => s
            .WithApplicability<EmptyRevenue>(x => !(x.Identity.ValuationApproach == ValuationApproaches.PAA && x.GetStorage().DataNodeDataBySystemName[x.Identity.DataNode].LiabilityType == LiabilityTypes.LRC)));

    protected string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
    private int timeStep => GetStorage().GetTimeStep(Identity.ProjectionPeriod); 

    IEnumerable<IfrsVariable> Revenue => GetScope<PremiumRevenue>(Identity).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                          EconomicBasis = x.EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition });
    
    private bool hasAmortizationStep => Revenue.Where(iv => iv.Values != null).Any(iv => Math.Abs(iv.Values.GetValidElement(Identity.ProjectionPeriod)) > Precision);

    IEnumerable<IfrsVariable> RevenueAmFactor =>  Identity.AocType == AocTypes.AM && hasAmortizationStep
        ? GetScope<DiscountedAmortizationFactorForRevenues>(Identity, o => o.WithContext(EconomicBasis)).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = EstimateTypes.F,
                          EconomicBasis = EconomicBasis,
                          DataNode = x.Identity.DataNode,
                          AocType = x.Identity.AocType,
                          Novelty = x.Identity.Novelty,
                          AmountType = x.EffectiveAmountType,
                          Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                          Partition = GetStorage().TargetPartition })
        : Enumerable.Empty<IfrsVariable>();
}
public interface EmptyRevenue : RevenueToIfrsVariable{
    IEnumerable<IfrsVariable> RevenueToIfrsVariable.Revenue => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> RevenueToIfrsVariable.RevenueAmFactor => Enumerable.Empty<IfrsVariable>();
}

## Experience Adj. for Premium

In [0]:
public interface EaForPremiumToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
  private string EconomicBasis => GetStorage().GetEconomicBasisDriver(Identity.DataNode);
  IEnumerable<IfrsVariable> BeEAForPremium => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC || Identity.IsReinsurance
    ? Enumerable.Empty<IfrsVariable>()
    : GetScope<BeExperienceAdjustmentForPremium>(Identity, o => o.WithContext(EconomicBasis)).RepeatOnce()
      .Select(sc => new IfrsVariable{ EstimateType = sc.EstimateType, 
                                      DataNode = sc.Identity.DataNode, 
                                      AocType = sc.Identity.AocType, 
                                      Novelty = sc.Identity.Novelty, 
                                      EconomicBasis = sc.EconomicBasis,
                                      AmountType = sc.AmountType,
                                      Values = SetProjectionValue(sc.Value, sc.Identity.ProjectionPeriod),
                                      Partition = sc.GetStorage().TargetPartition });
    
  IEnumerable<IfrsVariable> ActEAForPremium => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC || Identity.IsReinsurance
    ? Enumerable.Empty<IfrsVariable>()
    : GetScope<ActualExperienceAdjustmentOnPremium>(Identity).RepeatOnce()
        .Select(sc => new IfrsVariable{ EstimateType = sc.EstimateType, 
                         DataNode = sc.Identity.DataNode, 
                         AocType = sc.Identity.AocType, 
                         Novelty = sc.Identity.Novelty, 
                         AmountType = sc.AmountType,
                         Values = SetProjectionValue(sc.Value, sc.Identity.ProjectionPeriod),
                         Partition = GetStorage().TargetPartition });
}

## Technical margin

In [0]:
public interface TmToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<TmToIfrsVariable>(s => s.WithApplicability<EmptyTmIfrsVariable>(x => ComputationHelper.AocStepWithNoCsm.Contains(new AocStep(x.Identity.AocType, x.Identity.Novelty))));

    private string economicBasis => Identity.ValuationApproach == ValuationApproaches.VFA ? EconomicBases.C : EconomicBases.L;
    private IEnumerable<string> amountTypesForTm => GetScope<TechnicalMarginAmountType>((Identity, EstimateTypes.C)).Values;
    private bool hasTechnicalMargin => GetStorage().ImportFormat switch {
        ImportFormats.Cashflow => GetStorage().GetRawVariables(Identity.DataNode).Any(x => x.EstimateType == EstimateTypes.RA || 
            (x.EstimateType == EstimateTypes.BE && amountTypesForTm.Contains(x.AmountType))),
        _ => GetStorage().GetIfrsVariables(Identity.DataNode).Any(x => !GetStorage().EstimateTypesByImportFormat[ImportFormats.Actual].Contains(x.EstimateType) && 
            amountTypesForTm.Contains(x.AmountType))
    };

    IEnumerable<IfrsVariable> Csms => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC || Identity.ValuationApproach == ValuationApproaches.PAA
        ? Enumerable.Empty<IfrsVariable>()
        : GetScope<ContractualServiceMargin>(Identity).RepeatOnce()
          .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
               DataNode = x.Identity.DataNode,
               AocType = x.Identity.AocType,
               Novelty = x.Identity.Novelty,
               EconomicBasis = economicBasis,
               Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
               Partition = GetStorage().TargetPartition
            });

     IEnumerable<IfrsVariable> Loss => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC ||
                                       (Identity.ValuationApproach == ValuationApproaches.PAA && !hasTechnicalMargin )
        ? Enumerable.Empty<IfrsVariable>()
        : Identity.IsReinsurance 
           ? GetScope<LossRecoveryComponent>(Identity).RepeatOnce()
               .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                              DataNode = x.Identity.DataNode,
                                              AocType = x.Identity.AocType,
                                              Novelty = x.Identity.Novelty,
                                              EconomicBasis = economicBasis,
                                              Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                              Partition = GetStorage().TargetPartition
                                           })
           : GetScope<LossComponent>(Identity).RepeatOnce()
               .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                              DataNode = x.Identity.DataNode,
                                              AocType = x.Identity.AocType,
                                              Novelty = x.Identity.Novelty,
                                              EconomicBasis = economicBasis,
                                              Values = SetProjectionValue(x.Value, x.Identity.ProjectionPeriod),
                                              Partition = GetStorage().TargetPartition
                                           });
    
    IEnumerable<IfrsVariable> AmortizationFactor =>  Identity.AocType == AocTypes.AM && Loss.Concat(Csms).Where(x => x.Values != null).Any(x => Math.Abs(x.Values.GetValidElement(Identity.ProjectionPeriod)) > Precision)
        && GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LRC && 
        (Identity.ValuationApproach != ValuationApproaches.PAA || hasTechnicalMargin )
        ? GetScope<CurrentPeriodAmortizationFactor>((Identity, AmountTypes.CU, 0), o => o.WithContext(economicBasis)).RepeatOnce()
            .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                           DataNode = x.Identity.Id.DataNode,
                                           AocType = x.Identity.Id.AocType,
                                           Novelty = x.Identity.Id.Novelty,
                                           AmountType = x.EffectiveAmountType,
                                           EconomicBasis = x.EconomicBasis,
                                           Values = SetProjectionValue(x.Value, x.Identity.Id.ProjectionPeriod),
                                           Partition = GetStorage().TargetPartition
                                           })
        : Enumerable.Empty<IfrsVariable>();
}

public interface EmptyTmIfrsVariable: TmToIfrsVariable {
    IEnumerable<IfrsVariable> TmToIfrsVariable.Csms => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> TmToIfrsVariable.Loss => Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> TmToIfrsVariable.AmortizationFactor => Enumerable.Empty<IfrsVariable>();
}